In [1]:
!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install wurlitzer pandas duckdb==0.9.2


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import duckdb
from wurlitzer import pipes
from utils import generate_sql
from llama_cpp import Llama

In [4]:
with pipes() as (out, err):
    client = Llama(
        model_path="DuckDB-NSQL-7B-v0.1-q8_0.gguf",
        n_ctx=2048,
    )

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from DuckDB-NSQL-7B-v0.1-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32      

In [14]:
con = duckdb.connect("nyc.duckdb")

In [19]:
question = "get most recent tpep_pickup_datetime from taxi table starting with 'a'"

In [20]:
%%time
sql_query = generate_sql(question, con, client)

Llama.generate: 776 prefix-match hit, remaining 36 prompt tokens to eval

llama_print_timings:        load time =   10565.50 ms
llama_print_timings:      sample time =       0.75 ms /    31 runs   (    0.02 ms per token, 41333.33 tokens per second)
llama_print_timings: prompt eval time =    4569.30 ms /    36 tokens (  126.92 ms per token,     7.88 tokens per second)
llama_print_timings:        eval time =    3067.45 ms /    30 runs   (  102.25 ms per token,     9.78 tokens per second)
llama_print_timings:       total time =    7648.30 ms /    66 tokens


 SELECT tpep_pickup_datetime FROM taxi
ORDER BY tpep_pickup_datetime DESC
LIMIT 1;
CPU times: user 20.7 s, sys: 4.04 s, total: 24.7 s
Wall time: 7.78 s


In [21]:
sql_query

' SELECT tpep_pickup_datetime FROM taxi\nORDER BY tpep_pickup_datetime DESC\nLIMIT 1;'

In [22]:
con.execute(sql_query).fetchdf()

,tpep_pickup_datetime
0,2022-11-04 02:01:06
